In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256,(3,3),input_shape=[50,50,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(64,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(4,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [2]:
model.load_weights('../res/Models/modelAgeGender.h5')

In [3]:
import cv2, pickle
import dlib
import numpy as np

In [4]:
cam = cv2.VideoCapture('../res/test_data/videos/smile.mp4')#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video'

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

face_detector = dlib.get_frontal_face_detector()
margin = 20

In [5]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results,label,category_dict[label],acc)

            cv2.rectangle(feed, (x1, y1), (x2, y2),(0, 255, 0), 2)
            cv2.rectangle(feed, (x1-1, y1-20), (x1+99, y1), (0, 255, 0), -1)
            #cv2.putText(feed, "Face", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0 ,0, 255), 2)

            cv2.putText(feed, category_dict[label], (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
            cv2.putText(feed, str(acc) + '%', (x1 + 400, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    except:
        pass
    
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()

[[2.4007838e-06 8.8375187e-01 5.9970911e-20 1.1624573e-01]] 1 27-40 88
[[3.1752677e-06 8.7675273e-01 7.8096241e-20 1.2324410e-01]] 1 27-40 87
[[3.5774619e-06 8.8011867e-01 1.1284512e-19 1.1987782e-01]] 1 27-40 88
[[3.6993238e-06 8.4000456e-01 1.3603005e-19 1.5999174e-01]] 1 27-40 84
[[3.0541032e-06 7.8078014e-01 1.6220081e-19 2.1921682e-01]] 1 27-40 78
[[3.178117e-06 7.790533e-01 7.591464e-20 2.209436e-01]] 1 27-40 77
[[2.8270813e-06 7.6812619e-01 5.5319823e-20 2.3187098e-01]] 1 27-40 76
[[2.3862387e-06 7.8557628e-01 6.9165327e-20 2.1442139e-01]] 1 27-40 78
[[6.00208864e-07 7.72456944e-01 1.07484805e-19 2.27542445e-01]] 1 27-40 77
[[5.9762129e-07 7.3737514e-01 1.4911673e-19 2.6262423e-01]] 1 27-40 73
[[8.1839914e-07 7.8607470e-01 9.9739660e-19 2.1392448e-01]] 1 27-40 78
[[4.4831295e-06 9.8085487e-01 1.0329889e-17 1.9140739e-02]] 1 27-40 98
[[4.8622524e-06 9.7977579e-01 9.6035884e-18 2.0219306e-02]] 1 27-40 97
[[1.2922740e-05 9.4305128e-01 3.4761256e-17 5.6935783e-02]] 1 27-40 94
[[1.10

[[1.3851881e-01 8.1006300e-01 1.4741215e-11 5.1418189e-02]] 1 27-40 81
[[1.2120582e-01 8.4891671e-01 7.7967138e-12 2.9877372e-02]] 1 27-40 84
[[6.3663118e-02 9.3009615e-01 1.2448984e-11 6.2407353e-03]] 1 27-40 93
[[2.0390742e-01 7.9575640e-01 6.1436807e-11 3.3618149e-04]] 1 27-40 79
[[2.8581658e-01 7.1396792e-01 9.8015042e-12 2.1549745e-04]] 1 27-40 71
[[6.4621693e-01 3.5370076e-01 1.3240666e-10 8.2288483e-05]] 0 13-26 64
[[5.2364606e-01 4.7632203e-01 3.9193562e-08 3.1865646e-05]] 0 13-26 52
[[2.3461229e-01 7.6533198e-01 2.3202104e-07 5.5478671e-05]] 1 27-40 76
[[4.4436368e-01 5.5559605e-01 4.2667202e-07 3.9795930e-05]] 1 27-40 55
[[5.2599585e-01 4.7394758e-01 2.4366383e-07 5.6317185e-05]] 0 13-26 52
[[5.2229917e-01 4.7764438e-01 1.8896854e-07 5.6288809e-05]] 0 13-26 52
[[6.3729447e-01 3.6264911e-01 2.7369946e-08 5.6418845e-05]] 0 13-26 63
[[1.8727805e-01 8.1240910e-01 1.2769387e-08 3.1282796e-04]] 1 27-40 81
[[2.4336885e-01 7.5588804e-01 4.7390030e-09 7.4314099e-04]] 1 27-40 75
[[6.15